In [18]:
from omegaconf import OmegaConf

config = {
    'dataset': 'saier/unarxive_citrec',
    'n_train': 10_000,
    'n_valid': 1_000,
    'n_test': 1_000,
    'max_chars_len': 512,
    'min_chars_len': 128,
    'save_dir': '../../data/raw/unarxive_citrec/'
}
config = OmegaConf.create(config)

In [19]:
from datasets import load_dataset
from tqdm import tqdm

# Load the dataset in streaming mode
dataset = load_dataset(config.dataset, split='train', streaming=True)

def take_n_samples(n: int, split: str, batch_size: int = 250) -> list:
    dataset = load_dataset(config.dataset, split=split, streaming=True)
    samples = []
    bar = tqdm(total=n)
    for sample in dataset:
        if config.min_chars_len <= len(sample['text']) <= config.max_chars_len:
            samples.append(sample)
            bar.update(1)
        if len(samples) == n:
            break

    return samples

train_samples = take_n_samples(config.n_train, split='train')
valid_samples = take_n_samples(config.n_valid, split='validation')
test_samples = take_n_samples(config.n_test, split='test')

52792it [04:47, 39.63it/s]                           

KeyboardInterrupt: 

In [ ]:
print(f'Train samples: {len(train_samples)}')

Train samples: 10000


In [ ]:
def extract_texts(samples):
    return [sample['text'] for sample in samples]

train_texts = extract_texts(train_samples)
print(f'Train texts: {len(train_texts)}')
valid_texts = extract_texts(valid_samples)
test_texts = extract_texts(test_samples)

Train texts: 10000


In [ ]:
len(set(train_texts))

69

In [ ]:
def remove_overlap(texts_a, texts_b):
    overlap = list(set(texts_a) & set(texts_b))
    print(f'Overlap: {len(overlap)}')

    # remove
    texts_a = list(set(texts_a) - set(overlap))
    texts_b = list(set(texts_b) - set(overlap))
    return texts_a, texts_b

train_texts, valid_texts = remove_overlap(train_texts, valid_texts)
train_texts, test_texts = remove_overlap(train_texts, test_texts)
valid_texts, test_texts = remove_overlap(valid_texts, test_texts)

print(f"Train: {len(train_texts)}, Valid: {len(valid_texts)}, Test: {len(test_texts)}")

Overlap: 4
Before: 10000, 1065
After: 65, 49
Overlap: 0
Before: 65, 1023
After: 65, 25
Overlap: 4
Before: 49, 25
After: 45, 21
Train: 65, Valid: 45, Test: 21


In [ ]:
# assert that texts don't overlap
assert len(set(train_texts) & set(valid_texts)) == 0
assert len(set(train_texts) & set(test_texts)) == 0
assert len(set(valid_texts) & set(test_texts)) == 0

In [ ]:
import numpy as np

train_lens = np.array([len(text) for text in train_texts])

# 0.25 quantile, 0.5 quantile, 0.75 quantile
print("0.25, 0.5, 0.75 quantile:", np.quantile(train_lens, [0.25, 0.5, 0.75]))
print("Max len:", np.max(train_lens))
print("Min len:", np.min(train_lens))
print("Example:", train_texts[np.random.randint(0, len(train_texts))])

0.25, 0.5, 0.75 quantile: [290. 369. 460.]
Max len: 508
Min len: 145
Example: We will now see how the negative entropy \(h\)  yields
explicit steps that effectively enforce the simplicial constraints.
Indeed, after a short classical calculation (see, for instance [1]}),
it is possible to show that the update (REF ) is equivalent to the
following one: for all \(j\)  in \(\lbrace 1, \ldots , d\rbrace \) ,
\( \mathbf {z}_j^{k+1} = \frac{\mathbf {z}_j^k e^{-\eta _k \nabla f(\mathbf {z}^k)_j}}{\sum _{l=1}^d \mathbf {z}_l^k e^{-\eta _k \nabla f(\mathbf {z}^k)_l}},\) 



In [ ]:
import os
import json

os.makedirs(config.save_dir, exist_ok=True)
for texts, split_name in [
    (train_texts, 'train'),
    (valid_texts, 'valid'),
    (test_texts, 'test')
]:
    path = os.path.join(config.save_dir, split_name + '.json')
    with open(path, 'w') as f:
        json.dump(texts, f, indent=4, ensure_ascii=False)